In [1]:
from pymongo import MongoClient
from bson.objectid import ObjectId
import csv
import pandas as pd

In [2]:
conn = MongoClient('localhost', 27017)
db = conn.forum

In [3]:
cursor = db['cf_score'].find({},{'_id':0})

In [4]:
info = [a for a in cursor]

In [117]:
info[0]

{'board': 'dog-breeds',
 'member_id': '4337',
 'member_name': 'Dakota',
 'scores': 2.0,
 'title': '“Alaska, where women win the Iditarod and men mush poodles!”',
 'title_id': '2845'}

In [5]:
df = pd.DataFrame.from_dict(info, orient = 'columns')

In [6]:
df = df.dropna()

In [7]:
df.scores = df.scores.astype('int32')

In [8]:
df.member_id = df.member_id.astype('int32')

In [9]:
df.title_id = df.title_id.astype('int32')

In [11]:
df.count()

board          259658
member_id      259658
member_name    259658
scores         259658
title          259658
title_id       259658
dtype: int64

In [126]:
df[:3]

,board,member_id,member_name,scores,title,title_id
0,dog-breeds,4337,Dakota,2,"“Alaska, where women win the Iditarod and men ...",2845
1,dog-breeds,1674,alfi,5,"“Alaska, where women win the Iditarod and men ...",2845
2,dog-breeds,1198,Mikey,1,"“Alaska, where women win the Iditarod and men ...",2845


In [13]:
df2 = df[['member_id','member_name','title_id', 'title','scores']]

In [128]:
df2.max()

member_id                                                 211834
member_name                                               zynnox
title_id                                                  353946
title          “Alaska, where women win the Iditarod and men ...
scores                                                       281
dtype: object

In [14]:
len(df2)

259658

In [17]:
df3 = df2.drop_duplicates()

In [18]:
df3.count()

member_id      259658
member_name    259658
title_id       259658
title          259658
scores         259658
dtype: int64

In [15]:
m_id = df2[['member_id']].drop_duplicates()

In [16]:
m_id.count()

member_id    20500
dtype: int64

In [132]:
m_id[:3]

,member_id
0,4337
1,1674
2,1198


In [133]:
m_id.max()

member_id    211834
dtype: int32

In [136]:
m_id['m_id'] = m_id['member_id'].rank(ascending=True).astype('int32')

In [137]:
m_id.max()

member_id    211834
m_id          20500
dtype: int32

In [19]:
t_id = df2[['title_id']].drop_duplicates()

In [20]:
len(t_id)

45677

In [140]:
t_id.max()

title_id    353946
dtype: int32

In [141]:
t_id['t_id'] = t_id['title_id'].rank(ascending=True).astype('int32')

In [142]:
t_id.max()

title_id    353946
t_id         45677
dtype: int32

In [143]:
df2 = pd.merge(df2, m_id, how='left',
        left_on='member_id', right_on='member_id')

In [144]:
df2 = pd.merge(df2, t_id, how='left',
        left_on='title_id', right_on='title_id')

In [145]:
df2.max()

member_id                                                 211834
member_name                                               zynnox
title_id                                                  353946
title          “Alaska, where women win the Iditarod and men ...
scores                                                       281
m_id                                                       20500
t_id                                                       45677
dtype: object

In [147]:
collection = db['cf_scores']
for jdict in df2.to_dict(orient='records'):
    collection.insert_one(jdict)